In [46]:
import pandas as pd
import pandas as pd
import numpy as np

def calculate_basketball_percentages(df):
    """
    Calculate basketball percentage statistics from raw totals.
    
    Args:
        df (pandas.DataFrame): DataFrame containing the raw totals columns
        
    Returns:
        pandas.DataFrame: DataFrame with added percentage columns
    """
    # Make a copy to avoid modifying the original
    result = df.copy()
    
    # Basic shooting percentages
    result['Fg3Pct'] = (result['FG3M'] / result['FG3A'] * 100).fillna(0)
    result['Fg2Pct'] = (result['FG2M'] / result['FG2A'] * 100).fillna(0)
    result['NonHeaveFg3Pct'] = (result['FG3M'] / (result['FG3A']-result['HeaveAttempts']) * 100).fillna(0)
    
    # Advanced shooting percentages
    result['EfgPct'] = ((result['FG2M'] + 1.5 * result['FG3M']) / (result['FG2A'] + result['FG3A']) * 100).fillna(0)
    result['TsPct'] = (result['Points'] / (2 * (result['FG2A'] + result['FG3A'] + 0.44 * result['FTA'])) * 100).fillna(0)
    
    # Second chance percentages
    result['SecondChanceFg3Pct'] = (result['SecondChanceFG3M'] / result['SecondChanceFG3A'] * 100).fillna(0)
    result['SecondChanceFg2Pct'] = (result['SecondChanceFG2M'] / result['SecondChanceFG2A'] * 100).fillna(0)
    result['SecondChanceEfgPct'] = ((result['SecondChanceFG2M'] + 1.5 * result['SecondChanceFG3M']) / 
                                   (result['SecondChanceFG2A'] + result['SecondChanceFG3A']) * 100).fillna(0)
    result['SecondChanceTsPct'] = (result['SecondChancePoints'] / 
                                  (2 * (result['SecondChanceFG2A'] + result['SecondChanceFG3A'])) * 100).fillna(0)

    
    result['SecondChancePointsPct'] = (result['SecondChancePoints'] / result['Points'] * 100).fillna(0)
    
    # Shot distribution
    result['FG3APct'] = (result['FG3A'] / (result['FG2A'] + result['FG3A']) * 100).fillna(0)
    
    result['FG2APctBlocked'] = (result['Fg2aBlocked'] / result['FG2A'] * 100).fillna(0)
    result['AtRimPctBlocked'] = (result['BlockedAtRim'] / result['AtRimFGA'] * 100).fillna(0)
    result['LongMidRangePctBlocked'] = (result['BlockedLongMidRange'] / result['LongMidRangeFGA'] * 100).fillna(0)
    result['ShortMidRangePctBlocked'] = (result['BlockedShortMidRange'] / result['ShortMidRangeFGA'] * 100).fillna(0)
    result['FG3APctBlocked'] = (result['Fg3aBlocked'] / result['FG3A'] * 100).fillna(0)
    result['Corner3PctBlocked'] = (result['Blocked3s'] / result['Corner3FGA'] * 100).fillna(0)
    result['Arc3PctBlocked'] = (result['BlockedArc3'] / result['Arc3FGA'] * 100).fillna(0)
    
    # Rebound percentages - Field Goals (corrected to be relative to missed shots)
    result['DefFGReboundPct'] = (result['DefRebounds'] / (result['FG2A'] - result['FG2M'] + result['FG3A'] - result['FG3M']) * 100).fillna(0)
    result['OffFGReboundPct'] = (result['OffRebounds'] / (result['FG2A'] - result['FG2M'] + result['FG3A'] - result['FG3M']) * 100).fillna(0)
    
    # Rebound percentages by shot location (corrected to be relative to missed shots of that type)
    result['OffLongMidRangeReboundPct'] = (result['OffTwoPtRebounds'] / (result['LongMidRangeFGA'] - result['LongMidRangeFGM']) * 100).fillna(0)
    result['DefLongMidRangeReboundPct'] = (result['DefTwoPtRebounds'] / (result['LongMidRangeFGA'] - result['LongMidRangeFGM']) * 100).fillna(0)
    result['DefArc3ReboundPct'] = (result['DefThreePtRebounds'] / (result['Arc3FGA'] - result['Arc3FGM']) * 100).fillna(0)
    result['OffArc3ReboundPct'] = (result['OffThreePtRebounds'] / (result['Arc3FGA'] - result['Arc3FGM']) * 100).fillna(0)
    result['DefAtRimReboundPct'] = (result['DefTwoPtRebounds'] / (result['AtRimFGA'] - result['AtRimFGM']) * 100).fillna(0)
    result['OffAtRimReboundPct'] = (result['OffTwoPtRebounds'] / (result['AtRimFGA'] - result['AtRimFGM']) * 100).fillna(0)
    result['DefShortMidRangeReboundPct'] = (result['DefTwoPtRebounds'] / (result['ShortMidRangeFGA'] - result['ShortMidRangeFGM']) * 100).fillna(0)
    result['OffShortMidRangeReboundPct'] = (result['OffTwoPtRebounds'] / (result['ShortMidRangeFGA'] - result['ShortMidRangeFGM']) * 100).fillna(0)
    result['DefCorner3ReboundPct'] = (result['DefThreePtRebounds'] / (result['Corner3FGA'] - result['Corner3FGM']) * 100).fillna(0)
    result['OffCorner3ReboundPct'] = (result['OffThreePtRebounds'] / (result['Corner3FGA'] - result['Corner3FGM']) * 100).fillna(0)
    
    # Free throw rebound percentages (corrected to be relative to FT misses)
    result['DefFTReboundPct'] = (result['FTDefRebounds'] / (result['FTA'] - result['FtPoints']) * 100).fillna(0)
    result['OffFTReboundPct'] = (result['FTOffRebounds'] / (result['FTA'] - result['FtPoints']) * 100).fillna(0)
    
    # Rest of the
    result['DefFTReboundPct'] = (result['FTDefRebounds'] / (result['FTDefRebounds'] + result['FTOffRebounds']) * 100).fillna(0)
    result['OffFTReboundPct'] = (result['FTOffRebounds'] / (result['FTDefRebounds'] + result['FTOffRebounds']) * 100).fillna(0)
    
    # Assist percentages
    result['Assisted2sPct'] = (result['PtsAssisted2s'] / (2 * result['FG2M']) * 100).fillna(0)
    result['Assisted3sPct'] = (result['PtsAssisted3s'] / (3 * result['FG3M']) * 100).fillna(0)
    result['NonPutbacksAssisted2sPct'] = (result['PtsAssisted2s'] / (2 * (result['FG2M'] - result['PtsPutbacks']/2)) * 100).fillna(0)
    result['Corner3PctAssisted'] = (result['Corner3Assists'] / result['Corner3FGM'] * 100).fillna(0)
    result['Arc3PctAssisted'] = (result['Arc3Assists'] / result['Arc3FGM'] * 100).fillna(0)
    result['SecondChanceCorner3PctAssisted'] = (result['Corner3Assists'] / result['SecondChanceCorner3FGM'] * 100).fillna(0)
    result['SecondChanceArc3PctAssisted'] = (result['Arc3Assists'] / result['SecondChanceArc3FGM'] * 100).fillna(0)
    result['SecondChanceAtRimPctAssisted'] = (result['AtRimAssists'] / result['SecondChanceAtRimFGM'] * 100).fillna(0)
    result['AtRimPctAssisted'] = (result['AtRimAssists'] / result['AtRimFGM'] * 100).fillna(0)
    result['ShortMidRangePctAssisted'] = (result['ShortMidRangeAssists'] / result['ShortMidRangeFGM'] * 100).fillna(0)
    result['LongMidRangePctAssisted'] = (result['LongMidRangeAssists'] / result['LongMidRangeFGM'] * 100).fillna(0)
    
    # Penalty percentages
    result['PenaltyPointsPct'] = (result['PenaltyPoints'] / result['Points'] * 100).fillna(0)
    result['PenaltyOffPossPct'] = (result['PenaltyOffPoss'] / result['OffPoss'] * 100).fillna(0)
    result['PenaltyFg2Pct'] = (result['PenaltyFG2M'] / result['PenaltyFG2A'] * 100).fillna(0)
    result['PenaltyFg3Pct'] = (result['PenaltyFG3M'] / result['PenaltyFG3A'] * 100).fillna(0)
    result['PenaltyEfgPct'] = ((result['PenaltyFG2M'] + 1.5 * result['PenaltyFG3M']) / 
                              (result['PenaltyFG2A'] + result['PenaltyFG3A']) * 100).fillna(0)
    result['PenaltyTsPct'] = (result['PenaltyPoints'] / 
                             (2 * (result['PenaltyFG2A'] + result['PenaltyFG3A'] + 0.44 * result['FTA'])) * 100).fillna(0)
    
    # Miscellaneous percentages
    result['BlocksRecoveredPct'] = (result['RecoveredBlocks'] / result['Blocks'] * 100).fillna(0)
    result['LiveBallTurnoverPct'] = (result['LiveBallTurnovers'] / result['Turnovers'] * 100).fillna(0)
    result['SelfORebPct'] = (result['SelfOReb'] / result['OffRebounds'] * 100).fillna(0)
    
    # Fouls percentages
    total_shooting_fouls = result['TwoPtShootingFoulsDrawn'] + result['ThreePtShootingFoulsDrawn']
    result['ShootingFoulsDrawnPct'] = (result['ShootingFouls'] / (result['FG2A']+result['FG2A']) * 100).fillna(0)
    result['TwoPtShootingFoulsDrawnPct'] = ((result['TwoPtShootingFoulsDrawn'] )/ (result['FG2A'])* 100).fillna(0)
    result['ThreePtShootingFoulsDrawnPct'] = (result['ThreePtShootingFoulsDrawn'] / result['FG3A'] * 100).fillna(0)
    total_def_rebounds = result['DefTwoPtRebounds'] + result['DefThreePtRebounds']
    total_off_rebounds = result['OffTwoPtRebounds'] + result['OffThreePtRebounds']
    result['ThreePtShootingFoulsDrawnPct'] = result['ThreePtShootingFoulsDrawn'] / result['FG3A'] * 100
    
    result['DefTwoPtReboundPct'] = (result['DefTwoPtRebounds'] / total_def_rebounds * 100).fillna(0)
    result['DefThreePtReboundPct'] = (result['DefThreePtRebounds'] / total_def_rebounds * 100).fillna(0)
    result['OffTwoPtReboundPct'] = (result['OffTwoPtRebounds'] /(result['FG2A']-result['FG2M']) * 100).fillna(0)
    result['OffThreePtReboundPct'] = (result['OffThreePtRebounds'] / total_off_rebounds * 100).fillna(0)
    return result
df = pd.read_csv('data/2016/1610612739_ps.csv')
print(df.Minutes.sum())
df=df[df.EntityId.str.contains('2544')]
df.fillna(0,inplace=True)
comp = pd.read_csv('../../contract/nba_rapm/on-off/players_ps/2544.csv')
comp2 = pd.read_csv('../../contract/nba_rapm/on-off/players_ps/2544vs.csv')
comp=pd.concat([comp,comp2])
comp=comp[comp.year==2016]
print(comp.Minutes.sum())

comp=comp[comp.player_on==True]

comp['PlusMinus']

1004.0
2016.0


20    209.0
20   -209.0
Name: PlusMinus, dtype: float64

In [47]:
df.groupby('TeamId').sum(numeric_only=True)['PlusMinus']

TeamId
1610612739    209.0
Name: PlusMinus, dtype: float64

In [48]:
id_col=['EntityId',
 'TeamId',
 'Name',
 'ShortName',
 'RowId',
 'TeamAbbreviation',
'team_id',
 'year',
 'season',
 'team_vs']

In [49]:
pct= [col for col in df.columns if 'pct' in col.lower()]
sum = [col for col in df.columns if col not in id_col and col not in pct]
sum
sums= df.groupby('TeamId').sum(numeric_only=True)[sum].reset_index(drop=True)
exclude = [
    "SecondChanceTsPct",

    "OffAtRimReboundPct",
    "OffShortMidRangeReboundPct",
    "OffLongMidRangeReboundPct",
    "OffArc3ReboundPct",
    "DefAtRimReboundPct",
    "DefShortMidRangeReboundPct",
    "DefLongMidRangeReboundPct",
    "DefArc3ReboundPct",
    "DefCorner3ReboundPct",

    "FG2APctBlocked",
    "AtRimPctBlocked",
    "ShortMidRangePctBlocked",
    "LongMidRangePctBlocked",
    "BlocksRecoveredPct",
    "FG3APctBlocked",
    "Arc3PctBlocked",
    "Corner3PctBlocked",
    "DefArc3ReboundPct",
 "DefCorner3ReboundPct"
 "SelfORebPct",
"OffFTReboundPct",
"OffCorner3ReboundPct",
    "PenaltyTsPct",
	"SecondChanceCorner3PctAssisted",
    "SecondChanceAtRimPctAssisted"
]
pct = [col for col in pct if col not in exclude]
pct = [col for col in pct if 'rebound' not in col.lower()]

In [50]:
len(pct)

29

In [51]:
newframe=calculate_basketball_percentages(sums)

In [52]:
newframe[pct[21:30]]

,SelfORebPct,ShortMidRangePctAssisted,ShootingFoulsDrawnPct,TwoPtShootingFoulsDrawnPct,PenaltyFg3Pct,SecondChanceArc3PctAssisted,LongMidRangePctAssisted,ThreePtShootingFoulsDrawnPct
0,11.016949,41.463415,7.245575,14.60177,40.594059,850.0,29.67033,1.346154


In [53]:
comp[pct[21:30]]

,SelfORebPct,ShortMidRangePctAssisted,ShootingFoulsDrawnPct,TwoPtShootingFoulsDrawnPct,PenaltyFg3Pct,SecondChanceArc3PctAssisted,LongMidRangePctAssisted,ThreePtShootingFoulsDrawnPct
20,0.039877,0.414634,0.091028,0.131737,0.405941,1.0,0.296703,0.013333
20,0.047128,0.402062,0.089116,0.131579,0.360656,1.0,0.445783,0.011538


In [54]:
for col in sum:
    if 'foul' in col.lower():
        print(col)

Fouls
FoulsDrawn
Offensive Fouls Drawn
ShootingFouls
Charge Fouls
NonShootingFoulsDrawn
PenaltyPointsExcludingTakeFouls
PenaltyOffPossExcludingTakeFouls
NonShootingPenaltyNonTakeFoulsDrawn
Loose Ball Fouls Drawn
BlockingFoulsDrawn
TwoPtShootingFoulsDrawn
Offensive Fouls
Loose Ball Fouls
NonShootingPenaltyNonTakeFouls
BlockingFouls
ThreePtShootingFoulsDrawn
Charge Fouls Drawn


In [20]:
for col in comp.columns:
    if 'and' in col.lower():

        print(col)

2pt And 1 Free Throw Trips
3pt And 1 Free Throw Trips
